In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split


In [2]:
df = pd.read_csv("Churn_Modelling.csv")
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
df.shape

(10000, 14)

In [4]:
df.isnull().sum()

RowNumber          0
CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

In [5]:
df.columns

Index(['RowNumber', 'CustomerId', 'Surname', 'CreditScore', 'Geography',
       'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'Exited'],
      dtype='object')

In [6]:
df.Geography.value_counts()

Geography
France     5014
Germany    2509
Spain      2477
Name: count, dtype: int64

In [7]:
labelencode = LabelEncoder()
df['Gender'] = labelencode.fit_transform(df.Gender)
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [8]:
oneHotEncode = OneHotEncoder()
geo_encode = oneHotEncode.fit_transform(df[['Geography']])
geo_encode

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

In [9]:
oneHotEncode.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [10]:
geo_encode_df = pd.DataFrame(geo_encode.toarray(), columns=oneHotEncode.get_feature_names_out(['Geography']))
geo_encode_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [11]:
df.drop(['RowNumber', 'CustomerId','Surname', 'Geography'], axis=1, inplace=True)

In [12]:
cdf = pd.concat([df, geo_encode_df], axis=1)
cdf.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [13]:
import pickle 

with open("label_encoder_gender.pkl", "wb") as file:
    pickle.dump(labelencode, file)

with open("one_hot_encoder_geo.pkl", "wb") as file:
    pickle.dump(oneHotEncode, file)

In [14]:
X = cdf.drop('Exited',axis=1)
y = cdf['Exited']

In [15]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [16]:
scaler = StandardScaler()

X_train_scaler = scaler.fit_transform(X_train)
X_test_scaler = scaler.fit_transform(X_test)

In [17]:
with open("scaler.pkl", "wb") as file:
    pickle.dump(scaler, file)

In [18]:
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

12

In [21]:
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train_scaler.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

In [23]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [24]:
opt = tensorflow.keras.optimizers.Adam(learning_rate=0.01)
loss = tensorflow.keras.losses.BinaryCrossentropy()

In [25]:
model.compile(optimizer=opt, loss=loss, metrics=['accuracy'])

In [26]:
log_his = 'logs/fit/'+ datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_his, histogram_freq=1)

early_stopping_callback = EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True)

In [27]:
history = model.fit(X_train_scaler, y_train, validation_data=(X_test_scaler, y_test), epochs=100,
                    callbacks=[early_stopping_callback, tensorboard_callback])

Epoch 1/100


250/250 [==============================] - 2s 2ms/step - loss: 0.3995 - accuracy: 0.8347 - val_loss: 0.3520 - val_accuracy: 0.8520
Epoch 2/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3531 - accuracy: 0.8543 - val_loss: 0.3497 - val_accuracy: 0.8620
Epoch 3/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3482 - accuracy: 0.8576 - val_loss: 0.3535 - val_accuracy: 0.8610
Epoch 4/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3415 - accuracy: 0.8596 - val_loss: 0.3495 - val_accuracy: 0.8590
Epoch 5/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3403 - accuracy: 0.8604 - val_loss: 0.3713 - val_accuracy: 0.8535
Epoch 6/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3383 - accuracy: 0.8615 - val_loss: 0.3388 - val_accuracy: 0.8550
Epoch 7/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3334 - accuracy: 0.8636 - val_loss: 0.3388 - val_accuracy: 0.85

In [28]:
model.save('model.h5')

d:\Udemy\projects\ANN_Classification_Churn\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [30]:
%load_ext tensorboard

In [38]:
%tensorboard --logdir logs/fit/log_history/

Reusing TensorBoard on port 6008 (pid 7456), started 0:00:10 ago. (Use '!kill 7456' to kill it.)